## SVO and Modifer Type grouped Zero-shot NLI models' evaluation
- Author: Sushma Akoju, email: sushmaakoju@arizona.edu

In [ ]:
# !pip install openpyxl --upgrade

In [ ]:
!pip install pycm==3.7
!pip install altair==5.0.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.6/597.6 kB 9.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.5/709.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import pandas as pd
import pycm
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
import altair as alt


### Connect to the Google drive
- for zero shot evaluation
- for finetuned NLI models evlaution

In [ ]:
import os
path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/annotated/combined/results/zero-shot"
output_path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/annotated/combined/results/fine-tuned-results"
assert os.path.exists(path), "Error"
df = pd.read_excel(os.path.join(path,"zero-shot-analysis-apr24.xlsx"))
#zero-shot-analysis-apr24.xlsx
df = df[df.columns[1:]]
df = df[['SICK_id', 'Premise', 'Hypothesis', 'Modifier',
       'Premise/Hypothesis/Both', 'Part of Premise/Hypothesis Modified',
       'Modifier Type', 'Ground Truth',  'CompressedGT','nli-deberta-v3-base',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pair-classification-decomposable-attention-elmo']]
df['Part of Premise/Hypothesis Modified'] = df['Part of Premise/Hypothesis Modified'].str.lower()
df['Modifier Type'] = df['Modifier Type'].str.lower()

print(df.columns)
df.head()

Index(['SICK_id', 'Premise', 'Hypothesis', 'Modifier',
       'Premise/Hypothesis/Both', 'Part of Premise/Hypothesis Modified',
       'Modifier Type', 'Ground Truth', 'CompressedGT', 'nli-deberta-v3-base',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pair-classification-decomposable-attention-elmo'],
      dtype='object')


,SICK_id,Premise,Hypothesis,Modifier,Premise/Hypothesis/Both,Part of Premise/Hypothesis Modified,Modifier Type,Ground Truth,CompressedGT,nli-deberta-v3-base,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli,pair-classification-decomposable-attention-elmo
0,90,a man is jumping into an empty pool,a man is jumping into a full pool,NONE,NONE,none,no modifiers,Alternation,Contradiction,Contradiction,Contradiction,FE
1,90,every man is jumping into an empty pool,a man is jumping into a full pool,every,Premise,subject,universal,Alternation,Contradiction,Contradiction,Contradiction,FE
2,90,a man is jumping into an empty pool,every man is jumping into a full pool,every,Hypothesis,subject,universal,Negation,Contradiction,Contradiction,Contradiction,FE
3,90,every man is jumping into an empty pool,every man is jumping into a full pool,every,Both,subject,universal,Negation,Contradiction,Contradiction,Contradiction,FE
4,90,some man is jumping into an empty pool,a man is jumping into a full pool,some,Premise,subject,existential,Alternation,Contradiction,Contradiction,Contradiction,FE


In [ ]:
modifier_types = df['Modifier Type'].unique()

#### Group by Quantifier and plot the Confusion Matrices for Zero shot results.
##### For 3 NLI models:
- 'nli-deberta-v3-base',
- 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
- 'pair-classification-decomposable-attention-elmo'

In [ ]:

def get_classification_report(ytrue, ypred, model_name, mtype="", svo=""):
  res = classification_report(ytrue, ypred, output_dict=True)
  print(classification_report(ytrue, ypred))
  precision_m = res['macro avg']['precision']
  recall_m = res['macro avg']['recall']
  f1_macro = res['macro avg']['f1-score']
  acc = res['accuracy']
  # print(res)
  # precision_m, recall_m = precision_score(ytrue, ypred, average="macro"), recall_score(ytrue, ypred, average="macro")
  #"precision_w":precision_w, "recall_w":recall_w, "acc_w":acc_w,
  return {"model_name":model_name, "modifier_type":mtype,"svo": svo,
          "f1_macro":f1_macro, "precision_m":precision_m, "recall_m":recall_m, "acc":acc}


In [ ]:
gt = 'CompressedGT'
model_name = 'nli-deberta-v3-base'
mtype_scores = []
for modifier_type in modifier_types:
  print("This Confusion Matrix stats is for Modifier Type %s for nli-deberta-v3-base"%(modifier_type))
  ytrue = df[df['Modifier Type'] == modifier_type]['CompressedGT'].to_list()
  ypred = df[df['Modifier Type'] == modifier_type][model_name].to_list()
  res = get_classification_report(ytrue, ypred, model_name,modifier_type, "")
  # print(res['precision_m'], res['recall_m'], res)
  print(precision_score(ytrue, ypred, average="macro"), recall_score(ytrue, ypred, average="macro"))

This Confusion Matrix stats is for Modifier Type no modifiers for nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.75      1.00      0.86         6
           FE       0.83      1.00      0.91         5
      Neutral       1.00      0.25      0.40         4

     accuracy                           0.80        15
    macro avg       0.86      0.75      0.72        15
 weighted avg       0.84      0.80      0.75        15

0.8611111111111112 0.75
This Confusion Matrix stats is for Modifier Type universal for nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.71      0.95      0.81        80
           FE       0.58      0.94      0.72        35
      Neutral       0.81      0.39      0.52        88
           RE       0.09      0.07      0.08        14

     accuracy                           0.66       217
    macro avg       0.55      0.59      0.53       217
 weighted avg       0.69      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

In [ ]:
gt = 'CompressedGT'
model_name = 'nli-deberta-v3-base'
mtype_scores = []
for modifier_type in modifier_types:
  print("This Confusion Matrix stats is for Modifier Type %s for nli-deberta-v3-base"%(modifier_type))
  ytrue = df[df['Modifier Type'] == modifier_type]['CompressedGT'].to_list()
  ypred = df[df['Modifier Type'] == modifier_type][model_name].to_list()
  res = get_classification_report(ytrue, ypred, model_name, modifier_type)
  mtype_scores.append(res)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

This Confusion Matrix stats is for Modifier Type no modifiers for nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.75      1.00      0.86         6
           FE       0.83      1.00      0.91         5
      Neutral       1.00      0.25      0.40         4

     accuracy                           0.80        15
    macro avg       0.86      0.75      0.72        15
 weighted avg       0.84      0.80      0.75        15

This Confusion Matrix stats is for Modifier Type universal for nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.71      0.95      0.81        80
           FE       0.58      0.94      0.72        35
      Neutral       0.81      0.39      0.52        88
           RE       0.09      0.07      0.08        14

     accuracy                           0.66       217
    macro avg       0.55      0.59      0.53       217
 weighted avg       0.69      0.66      0.63       217



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model_name = 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli'
for modifier_type in modifier_types:
  print("This Confusion Matrix stats is for Modifier Type %s for %s"%(modifier_type, model_name))
  ytrue = df[df['Modifier Type'] == modifier_type]['CompressedGT'].to_list()
  ypred = df[df['Modifier Type'] == modifier_type][model_name].to_list()
  res = get_classification_report(ytrue, ypred, model_name, modifier_type)
  mtype_scores.append(res)

This Confusion Matrix stats is for Modifier Type no modifiers for ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
               precision    recall  f1-score   support

Contradiction       0.83      0.83      0.83         6
           FE       1.00      0.80      0.89         5
      Neutral       0.50      0.50      0.50         4
           RE       0.00      0.00      0.00         0

     accuracy                           0.73        15
    macro avg       0.58      0.53      0.56        15
 weighted avg       0.80      0.73      0.76        15

This Confusion Matrix stats is for Modifier Type universal for ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
               precision    recall  f1-score   support

Contradiction       0.73      0.95      0.83        80
           FE       0.59      0.77      0.67        35
      Neutral       0.74      0.48      0.58        88
           RE       0.20      0.14      0.17        14

     accuracy                           0.68 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

In [ ]:
model_name = 'pair-classification-decomposable-attention-elmo'
for modifier_type in modifier_types:
  print("This Confusion Matrix stats is for Modifier Type %s for %s"%(modifier_type, model_name))
  ytrue = df[df['Modifier Type'] == modifier_type]['CompressedGT'].to_list()
  ypred = df[df['Modifier Type'] == modifier_type][model_name].to_list()
  res = get_classification_report(ytrue, ypred, model_name, modifier_type)
  mtype_scores.append(res)

This Confusion Matrix stats is for Modifier Type no modifiers for pair-classification-decomposable-attention-elmo
               precision    recall  f1-score   support

Contradiction       0.00      0.00      0.00         6
           FE       0.33      1.00      0.50         5
      Neutral       0.00      0.00      0.00         4

     accuracy                           0.33        15
    macro avg       0.11      0.33      0.17        15
 weighted avg       0.11      0.33      0.17        15

This Confusion Matrix stats is for Modifier Type universal for pair-classification-decomposable-attention-elmo


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

               precision    recall  f1-score   support

Contradiction       0.00      0.00      0.00        80
           FE       0.16      1.00      0.28        35
      Neutral       0.00      0.00      0.00        88
           RE       0.00      0.00      0.00        14

     accuracy                           0.16       217
    macro avg       0.04      0.25      0.07       217
 weighted avg       0.03      0.16      0.04       217

This Confusion Matrix stats is for Modifier Type existential for pair-classification-decomposable-attention-elmo
               precision    recall  f1-score   support

Contradiction       0.00      0.00      0.00        88
           FE       0.26      1.00      0.42        80
      Neutral       0.00      0.00      0.00       130
           RE       0.00      0.00      0.00         5

     accuracy                           0.26       303
    macro avg       0.07      0.25      0.10       303
 weighted avg       0.07      0.26      0.11       303

T

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
mtype_scores

[{'model_name': 'nli-deberta-v3-base',
  'modifier_type': 'no modifiers',
  'svo': '',
  'f1_macro': 0.7220779220779221,
  'precision_m': 0.8611111111111112,
  'recall_m': 0.75,
  'acc': 0.8},
 {'model_name': 'nli-deberta-v3-base',
  'modifier_type': 'universal',
  'svo': '',
  'f1_macro': 0.53332561300592,
  'precision_m': 0.5474151606714321,
  'recall_m': 0.5876623376623377,
  'acc': 0.663594470046083},
 {'model_name': 'nli-deberta-v3-base',
  'modifier_type': 'existential',
  'svo': '',
  'f1_macro': 0.5101121755506428,
  'precision_m': 0.5453261158048393,
  'recall_m': 0.5608391608391609,
  'acc': 0.6105610561056105},
 {'model_name': 'nli-deberta-v3-base',
  'modifier_type': 'negation',
  'svo': '',
  'f1_macro': 0.2678053830227743,
  'precision_m': 0.35558840436889216,
  'recall_m': 0.4637061476965677,
  'acc': 0.30538922155688625},
 {'model_name': 'nli-deberta-v3-base',
  'modifier_type': 'adjectives/adverbs',
  'svo': '',
  'f1_macro': 0.5768192023658427,
  'precision_m': 0.6087

#### F1, Precision, recall and Accuracy for Zero shot results. (Desiderata)
##### For 3 NLI models:
- 'nli-deberta-v3-base',
- 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
- 'pair-classification-decomposable-attention-elmo'

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
model_names = ['nli-deberta-v3-base',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pair-classification-decomposable-attention-elmo']
gt = 'CompressedGT'
scores = []
for model_name in model_names:
  print("This Confusion Matrix stats is for %s"%( model_name))
  ytrue = df['CompressedGT'].to_list()
  ypred = df[model_name].to_list()
  res = get_classification_report(ytrue, ypred, model_name)
  scores.append(res)

This Confusion Matrix stats is for nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.63      0.96      0.76       422
           FE       0.60      0.93      0.73       259
      Neutral       0.90      0.35      0.50       593
           RE       0.11      0.10      0.11        30

     accuracy                           0.66      1304
    macro avg       0.56      0.59      0.53      1304
 weighted avg       0.73      0.66      0.62      1304

This Confusion Matrix stats is for ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
               precision    recall  f1-score   support

Contradiction       0.63      0.84      0.72       422
           FE       0.67      0.80      0.73       259
      Neutral       0.69      0.44      0.54       593
           RE       0.07      0.13      0.10        30

     accuracy                           0.63      1304
    macro avg       0.52      0.55      0.52      1304
 weighted avg       0.65   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
scores

[{'model_name': 'nli-deberta-v3-base',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.5254181116194013,
  'precision_m': 0.5600705548236663,
  'recall_m': 0.5860073548632008,
  'acc': 0.6579754601226994},
 {'model_name': 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.5200345993764848,
  'precision_m': 0.5155704861587215,
  'recall_m': 0.5532624836671655,
  'acc': 0.6334355828220859},
 {'model_name': 'pair-classification-decomposable-attention-elmo',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.08285348688419705,
  'precision_m': 0.04965490797546012,
  'recall_m': 0.25,
  'acc': 0.19861963190184048}]

In [ ]:
scores

[{'model_name': 'nli-deberta-v3-base',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.5254181116194013,
  'precision_m': 0.5600705548236663,
  'recall_m': 0.5860073548632008,
  'acc': 0.6579754601226994},
 {'model_name': 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.5200345993764848,
  'precision_m': 0.5155704861587215,
  'recall_m': 0.5532624836671655,
  'acc': 0.6334355828220859},
 {'model_name': 'pair-classification-decomposable-attention-elmo',
  'modifier_type': '',
  'svo': '',
  'f1_macro': 0.08285348688419705,
  'precision_m': 0.04965490797546012,
  'recall_m': 0.25,
  'acc': 0.19861963190184048}]

##### Uncomment the following lines to save the evaluation results.

In [ ]:
pd.DataFrame(scores).to_csv(os.path.join(path, "zero_shot_scores.xlsx"))
pd.DataFrame(mtype_scores).to_csv(os.path.join(path, "zero_shot_modifier_scores.xlsx"))

#### F1, Precision, recall and accuracy - Grouped by Subject Verb Object (SVO) of the Zero shot analysis.
##### For 3 NLI models:
- 'nli-deberta-v3-base',
- 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
- 'pair-classification-decomposable-attention-elmo'

In [ ]:
svo = "Part of Premise/Hypothesis Modified"
df[svo] = df[svo].str.lower()

In [ ]:
model_names = ['nli-deberta-v3-base',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pair-classification-decomposable-attention-elmo']
gt = 'CompressedGT'
df[svo] = df[svo].astype(str)
svo_scores = []
for svo_type in df[svo].unique():
  for model_name in model_names:
    print("The scores are for SVO %s, model name %s"%( svo_type,model_name))
    ytrue = df[df[svo] == svo_type]['CompressedGT'].to_list()
    ypred = df[df[svo] == svo_type][model_name].to_list()
    res = get_classification_report(ytrue, ypred, model_name,"", svo_type)
    svo_scores.append(res)

The scores are for SVO none, model name nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.75      1.00      0.86         6
           FE       0.83      1.00      0.91         5
      Neutral       1.00      0.25      0.40         4

     accuracy                           0.80        15
    macro avg       0.86      0.75      0.72        15
 weighted avg       0.84      0.80      0.75        15

The scores are for SVO none, model name ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli
               precision    recall  f1-score   support

Contradiction       0.83      0.83      0.83         6
           FE       1.00      0.80      0.89         5
      Neutral       0.50      0.50      0.50         4
           RE       0.00      0.00      0.00         0

     accuracy                           0.73        15
    macro avg       0.58      0.53      0.56        15
 weighted avg       0.80      0.73      0.76        15

The scores are f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

               precision    recall  f1-score   support

Contradiction       0.00      0.00      0.00       182
           FE       0.19      1.00      0.32       106
      Neutral       0.00      0.00      0.00       257
           RE       0.00      0.00      0.00        15

     accuracy                           0.19       560
    macro avg       0.05      0.25      0.08       560
 weighted avg       0.04      0.19      0.06       560

The scores are for SVO verb, model name nli-deberta-v3-base
               precision    recall  f1-score   support

Contradiction       0.57      0.94      0.71        66
           FE       0.48      0.94      0.64        34
      Neutral       0.91      0.27      0.42       115
           RE       0.09      0.20      0.13         5

     accuracy                           0.57       220
    macro avg       0.51      0.59      0.47       220
 weighted avg       0.72      0.57      0.53       220

The scores are for SVO verb, model name ynie/roberta-l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
svo_scores

[{'model_name': 'nli-deberta-v3-base',
  'modifier_type': '',
  'svo': 'none',
  'f1_macro': 0.7220779220779221,
  'precision_m': 0.8611111111111112,
  'recall_m': 0.75,
  'acc': 0.8},
 {'model_name': 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
  'modifier_type': '',
  'svo': 'none',
  'f1_macro': 0.5555555555555556,
  'precision_m': 0.5833333333333334,
  'recall_m': 0.5333333333333333,
  'acc': 0.7333333333333333},
 {'model_name': 'pair-classification-decomposable-attention-elmo',
  'modifier_type': '',
  'svo': 'none',
  'f1_macro': 0.16666666666666666,
  'precision_m': 0.1111111111111111,
  'recall_m': 0.3333333333333333,
  'acc': 0.3333333333333333},
 {'model_name': 'nli-deberta-v3-base',
  'modifier_type': '',
  'svo': 'subject',
  'f1_macro': 0.5069673066246942,
  'precision_m': 0.5468556391924039,
  'recall_m': 0.5732266729917417,
  'acc': 0.6428571428571429},
 {'model_name': 'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
  'modifier_type': '',
  'svo': 'sub

##### Uncomment the following lines to save the SVO scores.

In [ ]:
pd.DataFrame(svo_scores).to_csv(os.path.join(path, "zero_shot_modified_svo_scores.xlsx"))

In [ ]:
# !pip install altair

In [ ]:
import altair as alt

source = pd.DataFrame.from_dict(scores)
source.head()

,model_name,modifier_type,svo,f1_macro,precision_m,recall_m,acc
0,nli-deberta-v3-base,,,0.525418,0.560071,0.586007,0.657975
1,ynie/roberta-large-snli_mnli_fever_anli_R1_R2_...,,,0.520035,0.515570,0.553262,0.633436
2,pair-classification-decomposable-attention-elmo,,,0.082853,0.049655,0.250000,0.198620


In [ ]:
source['model_name'].unique()

array(['nli-deberta-v3-base',
       'ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli',
       'pair-classification-decomposable-attention-elmo'], dtype=object)

In [ ]:
source = pd.DataFrame.from_dict(scores)
source['model_name'][source['model_name'] == "nli-deberta-v3-base"] = "deberta"
source['model_name'][source['model_name'] == "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]= "roberta"
source['model_name'][source['model_name'] == "pair-classification-decomposable-attention-elmo"] = "elmo"
# print(source.sort_values('f1'))
# sorted = source.sort_values('f1')

<ipython-input-26-4af710ceab66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source['model_name'][source['model_name'] == "nli-deberta-v3-base"] = "deberta"
<ipython-input-26-4af710ceab66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source['model_name'][source['model_name'] == "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]= "roberta"
<ipython-input-26-4af710ceab66>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sour

In [ ]:
# line1 = alt.Chart(sorted[sorted['model_name'] == 'deberta'][ ['f1']]).mark_rule().encode(y='f1', color='f1')
# line2 = alt.Chart(sorted[sorted['model_name'] == 'roberta'][ ['f1']]).mark_rule().encode(y='f1', color='f1')
# line3 = alt.Chart(sorted[sorted['model_name'] == 'elmo'][ ['f1']]).mark_rule().encode(y='f1', color='f1')

TypeError: ignored

In [ ]:
sorted[sorted['model_name'] == 'deberta'][['f1']]

In [ ]:
qsource = pd.DataFrame.from_dict(qscores)
qsource['model_name'][qsource['model_name'] == "nli-deberta-v3-base"] = "deberta"
qsource['model_name'][qsource['model_name'] == "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]= "roberta"
qsource['model_name'][qsource['model_name'] == "pair-classification-decomposable-attention-elmo"] = "elmo"
all = pd.concat([source, qsource])
all.head()

In [ ]:
import numpy as np
all['quantifier_type'] = all['quantifier_type'].fillna("All")
all['quantifier_type']

In [ ]:
all[(all['model_name'] == "deberta") &(all['quantifier_type'] != "All") ]

In [ ]:
deberta = all[(all['model_name'] == "deberta") &(all['quantifier_type'] != "All") ]
deberta

In [ ]:
lines = alt.Chart(all.sort_values('f1'),title="Scores for Zero-shot Analysis").mark_line().encode(
    x=alt.X('model_name:O',axis=alt.Axis(labelAngle=-45),sort='-y'),
    y=alt.Y('f1:Q'),
    color='quantifier_type:N',size=alt.value(1)
).properties(width=400, height = 200)

lines

#### Commented.

In [ ]:


# base = alt.Chart(all).properties(width=450, height = 200)

# points = alt.Chart(source.sort_values('f1'),title="Overall scores for Zero-shot Analysis").mark_point().encode(
#     x=alt.X('model_name:N',axis=alt.Axis(labelAngle=-45),sort='-y'),
#     y=alt.Y('f1:Q'),
#     color='model_name:N',size=alt.value(12)
# ).properties(width=400, height = 200)
# text = points.mark_text(
#     align='left',
#     baseline='middle',
#     dx=7
# ).encode(
#     text=alt.Text('f1', format=".2f")
# )

# line = base.mark_line().encode(
#     x='date',
#     y='price',
#     color='symbol'
# )
# rule = base.mark_rule().encode(
#     y='average(price)',
#     color='symbol',
#     size=alt.value(2)
# )

# rule = base.mark_rule().encode(
#     y='average(price)',
#     color='symbol',
#     size=alt.value(2)
# )

# line + rule

In [ ]:
points = alt.Chart(source.sort_values('f1'),title="Overall scores for Zero-shot Analysis").mark_point().encode(
    x=alt.X('model_name:N',axis=alt.Axis(labelAngle=-45),sort='-y'),
    y=alt.Y('f1:Q'),
    color='model_name:N',size=alt.value(12)
).properties(width=400, height = 200)
text = points.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text=alt.Text('f1', format=".2f")
)

points + text

In [ ]:
# alt.themes.enable('my_theme')
qtypes = all['quantifier_type'].to_list()
range_ = ['lightsteelblue','steelblue', 'lightgray','gray','dodgerblue', 'lightskyblue']
bars = alt.Chart(all.sort_values('f1')).mark_bar().encode(
    x=alt.X("model_name:N", axis=alt.Axis(labelAngle=-0)),
    y="f1:Q",
    xOffset="quantifier_type:N",
    color=alt.Color("quantifier_type:N").scale(domain=qtypes, range=range_)
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text=alt.Text('f1', format=".2f")
)

bars + text

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/annotated/combined/results/zero-shot
!ls

In [ ]:
# model_paths = [os.path.join(output_path) for model_name in ["nli-deberta-v3-base", "roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]]
# model_paths